In [29]:
import pickle
import pandas as pd
import os
import re

In [30]:
# Format:
# Bowler, batsman 
# Runs in numbers/words, out else 0. If wide/no ball then ignore entry
# out
def to_corrrect_format(filepath): 
    df = pd.read_csv(filepath, sep='delimiter',header=None, engine="python") #, names=["Players", "Outcome"]
    numbers_dict = {"zero": 0, "one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6}
    data = []
    wkt_taken = 0
    num_over = 0
    prev_bowler = ''
    for index, row in df.iterrows():
        line = row.to_string(index=False)
        list = line.split(",")

        bowler, batsman = list[0].split(" to ")    
        if prev_bowler!=bowler: # what about case senstivity?
            prev_bowler=bowler
            num_over+=1

        find_runs = r'(zero|one|two|three|four|five|six|0|1|2|3|4|5|6)'
        find_extra = r'(wide|no ball)'
        outcome = re.findall(find_runs, list[1], flags=re.IGNORECASE)
        if re.findall("out", list[-1], flags=re.IGNORECASE):
            wkt_taken += 1
            data.append([bowler, batsman, wkt_taken, num_over, 7])
        elif re.findall(find_extra, list[1], flags=re.IGNORECASE):
            continue
        elif outcome:
            # print(outcome)
            runs = 0
            if outcome[0].isdigit():
                runs = int(outcome[0])
            else:
                num = numbers_dict.get(outcome[0].lower(), None)
                if num is not None:
                    runs = num
            data.append([bowler, batsman, wkt_taken, num_over, runs])
        else:
            data.append([bowler, batsman, wkt_taken, num_over, 0])
            # print("yay!")
        # print(list)
    # print(data)
    new_path = filepath.split(".")
    with open(new_path[0]+'.pkl', 'wb') as file:
        pickle.dump(data, file)

In [25]:
# current issues:
#   - Same player last names
#   - Dismissal on extras leads to >6 balls in an over
#   - counting byes as runs

In [31]:
# df = pd.read_csv("T20_matches/australia-vs-bangladesh-11th-match-group-a-412688/textsInnings1.csv", sep='delimiter',header=None) #, names=["Players", "Outcome"]
# df.head()

to_corrrect_format("T20_matches/australia-vs-bangladesh-11th-match-group-a-412688/textsInnings1.csv")

In [32]:
rootdir = 'T20_matches'

for subdir, dirs, files in os.walk(rootdir):
    try: 
        for file in files:
            to_corrrect_format(os.path.join(subdir, file))
    except: 
        pass